In [1]:
import pandas as pd

df = pd.read_csv('/projects/diffusion_bootcamp/datasets/ibmaml/HI-Small_Trans.csv')
df.head()


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [7]:
print(len(set(df['Account'])))

print(df.shape)


print(len(set(df['Account.1'])))



496995
(5078345, 11)
420636


In [10]:
matching_rows = df[ (df['Account'] == df['Account.1'])  & (df['Is Laundering'] == 1) ]
print(len(matching_rows))
print(matching_rows)

11
                Timestamp  From Bank    Account  To Bank  Account.1  \
318130   2022/09/04 05:03      48309  811C599A0    48309  811C599A0   
887448   2022/09/01 14:04      15231  80266F880    15231  80266F880   
1982464  2022/09/03 19:55      15231  80266F880    15231  80266F880   
1984629  2022/09/04 21:37     150240  812D22980   150240  812D22980   
2616583  2022/09/05 15:41      23691  8021353D0    23691  8021353D0   
2757791  2022/09/09 09:08      48309  811C599A0    48309  811C599A0   
3006137  2022/09/06 23:04        119  812A09CF0      119  812A09CF0   
3240081  2022/09/10 23:17        119  811C597B0      119  811C597B0   
3732502  2022/09/11 05:36      48309  811C599A0    48309  811C599A0   
4958161  2022/09/17 17:23        118  811B6E170      118  811B6E170   
5038137  2022/09/15 13:55      48309  811C599A0    48309  811C599A0   

         Amount Received Receiving Currency  Amount Paid Payment Currency  \
318130          48649.42        Saudi Riyal     48649.42      Saudi

In [22]:
# Feature Engineering 1a: same account activity
# df["Same Account Activity"] = (df["Account"] == df["Account.1"]).astype(int)

# Feature Engineering 1b: identical account activity
df["Identitcal Account Activity"] = ((df["Account"] == df["Account.1"]) & (df["From Bank"] == df["To Bank"])).astype(int)


In [16]:

# Feature Engineering 2: same bank activity
# df["Same Bank Activity"] = (df['From Bank'] == df['To Bank']).astype(int)

In [42]:
# Feature Engineering 3: Timestamp normalization
df['Datetime'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')
# df['Datetime'].apply(lambda x: int(x.timestamp()))
df["SecondsSinceEpoch"] = (df['Datetime'].astype(int) // 10**9)


In [43]:
# Final DF
feature_engineered_df = df[["Timestamp", "SecondsSinceEpoch", "Identitcal Account Activity", "Amount Received", "Receiving Currency", "Amount Paid", "Payment Currency", "Payment Format", "Is Laundering"]]
feature_engineered_df

,Timestamp,SecondsSinceEpoch,Identitcal Account Activity,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,1661991600,1,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0
1,2022/09/01 00:20,1661991600,0,0.010000,US Dollar,0.010000,US Dollar,Cheque,0
2,2022/09/01 00:00,1661990400,1,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0
3,2022/09/01 00:02,1661990520,1,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0
4,2022/09/01 00:06,1661990760,1,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0
...,...,...,...,...,...,...,...,...,...
5078340,2022/09/10 23:57,1662854220,0,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0
5078341,2022/09/10 23:35,1662852900,0,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0
5078342,2022/09/10 23:52,1662853920,0,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0
5078343,2022/09/10 23:46,1662853560,0,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0
